## Results for Tables 1, 2, 3 for CARLA methods

In [1]:
from carla.data.catalog import OnlineCatalog
import pandas as pd
import os
import numpy as np

# load catalog dataset
data_name = "adult"
# data_name = "give_me_some_credit"
data_name = 'compas'
dataset = OnlineCatalog(data_name)


/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/nr/samba/

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [16]:
dataset.df.head(1)

,age,two_year_recid,priors_count,length_of_stay,score,c_charge_degree_M,race_Other,sex_Male
0,0.205128,1.0,0.0,0.01375,1,0.0,0.0,1.0


In [2]:
from carla.models.catalog import MLModelCatalog
import torch
torch.manual_seed(0)

ml_model = MLModelCatalog(
        dataset, 
        model_type="ann", 
        load_online=False, 
        backend="pytorch"
    )
if data_name == 'adult':
    ml_model.train(
    learning_rate=0.002,
    epochs=20,
    batch_size=1024,
    hidden_size=[18, 9, 3],
    force_train=True, # don't forget to add this or it might load an older model from disk
    )
elif data_name == 'give_me_some_credit':
    ml_model.train(
    learning_rate=0.002,
    epochs=20,
    batch_size=2048,
    hidden_size=[18, 9, 3],
    force_train=True, # don't forget to add this or it might load an older model from disk
    )
elif data_name == 'compas':
    ml_model.train(
    learning_rate=0.002,
    epochs=25,
    batch_size=25,
    hidden_size=[18, 9, 3],
    force_train=True, # don't forget to add this or it might load an older model from disk
    )


balance on test set 0.8131345863037374, balance on test set 0.8191834089436163
Epoch 0/24
----------


/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


train Loss: 0.5018 Acc: 0.7280

test Loss: 0.4022 Acc: 0.8192

Epoch 1/24
----------
train Loss: 0.4002 Acc: 0.8283

test Loss: 0.3704 Acc: 0.8471

Epoch 2/24
----------
train Loss: 0.3796 Acc: 0.8356

test Loss: 0.3557 Acc: 0.8419

Epoch 3/24
----------
train Loss: 0.3665 Acc: 0.8423

test Loss: 0.3405 Acc: 0.8477

Epoch 4/24
----------
train Loss: 0.3620 Acc: 0.8438

test Loss: 0.3483 Acc: 0.8464

Epoch 5/24
----------
train Loss: 0.3589 Acc: 0.8445

test Loss: 0.3409 Acc: 0.8542

Epoch 6/24
----------
train Loss: 0.3592 Acc: 0.8434

test Loss: 0.3361 Acc: 0.8555

Epoch 7/24
----------
train Loss: 0.3579 Acc: 0.8451

test Loss: 0.3358 Acc: 0.8529

Epoch 8/24
----------
train Loss: 0.3564 Acc: 0.8445

test Loss: 0.3367 Acc: 0.8548

Epoch 9/24
----------
train Loss: 0.3559 Acc: 0.8440

test Loss: 0.3383 Acc: 0.8516

Epoch 10/24
----------
train Loss: 0.3564 Acc: 0.8471

test Loss: 0.3534 Acc: 0.8542

Epoch 11/24
----------
train Loss: 0.3556 Acc: 0.8447

test Loss: 0.3376 Acc: 0.8535



In [4]:
from sklearn import metrics

if data_name == 'adult':
    y = dataset.df_test['income']
elif data_name == 'give_me_some_credit':
    y = dataset.df_test['SeriousDlqin2yrs']
elif data_name == 'compas':
    y = dataset.df_test['score']

pred = ml_model.predict_proba(dataset.df_test)
pred = [row[1] for row in pred]
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
metrics.auc(fpr, tpr)

0.861973140964566

In [5]:
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:100]

## Violation

In [6]:
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

results = []
for method in ['cchvae', 'cem-vae', 'revise', 'clue', 'crud', 'face']:
    if data_name == 'adult':
        cfs = pd.read_csv("Results/adult_manifold_results.csv")
    elif data_name == 'give_me_some_credit':
        cfs = pd.read_csv("Results/give_me_some_credit_manifold_results.csv")
    elif data_name == 'compas':
        cfs = pd.read_csv("Results/compas_manifold_results.csv")
    factuals = predict_negative_instances(ml_model, dataset.df)
    test_factual = factuals.iloc[:100]

    cfs.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
    cfs.set_index(['index'], inplace=True)

    df_cfs = cfs[cfs['method'] == method].drop(['method',	'data'], axis=1)

    nan_idx = df_cfs.index[df_cfs.isnull().any(axis=1)]

    output_factuals = test_factual.copy()
    output_counterfactuals = df_cfs.copy()

    output_factuals = output_factuals.drop(index=nan_idx)
    output_counterfactuals = output_counterfactuals.drop(index=nan_idx)

    test_factual = output_factuals
    df_cfs = output_counterfactuals

    df_decoded_cfs = dataset.inverse_transform(df_cfs.copy())

    df_factuals = dataset.inverse_transform(test_factual.copy())

    cfs_continuous_immutable = df_decoded_cfs[
        intersection(dataset.continuous, dataset.immutables)
    ]
    factual_continuous_immutable = df_factuals[
        intersection(dataset.continuous, dataset.immutables)
    ]

    continuous_violations = np.invert(
        np.isclose(cfs_continuous_immutable, factual_continuous_immutable)
    )
    continuous_violations = np.sum(continuous_violations, axis=1).reshape(
        (-1, 1)
    )  # sum over features

    # check categorical by boolean comparison
    cfs_categorical_immutable = df_decoded_cfs[
        intersection(dataset.categorical, dataset.immutables)
    ]
    factual_categorical_immutable = df_factuals[
        intersection(dataset.categorical, dataset.immutables)
    ]

    categorical_violations = cfs_categorical_immutable != factual_categorical_immutable
    categorical_violations = np.sum(categorical_violations.values, axis=1).reshape(
        (-1, 1)
    )  # sum over features

    total_violations = continuous_violations + categorical_violations

    for x in total_violations:
        results.append(x[0])
        
final_results = cfs.copy()
final_results.dropna(inplace=True)
final_results['violations'] = results

/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change 

## Distance

In [7]:
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

results = []
for method in ['cchvae', 'cem-vae', 'revise', 'clue', 'crud', 'face']:
    if data_name == 'adult':
        cfs = pd.read_csv("Results/adult_manifold_results.csv")
    elif data_name == 'give_me_some_credit':
        cfs = pd.read_csv("Results/give_me_some_credit_manifold_results.csv")
    elif data_name == 'compas':
        cfs = pd.read_csv("Results/compas_manifold_results.csv")
    factuals = predict_negative_instances(ml_model, dataset.df)
    test_factual = factuals.iloc[:100]

    cfs.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
    cfs.set_index(['index'], inplace=True)

    df_cfs = cfs[cfs['method'] == method].drop(['method',	'data'], axis=1)

    nan_idx = df_cfs.index[df_cfs.isnull().any(axis=1)]

    output_factuals = test_factual.copy()
    output_counterfactuals = df_cfs.copy()

    output_factuals = output_factuals.drop(index=nan_idx)
    output_counterfactuals = output_counterfactuals.drop(index=nan_idx)

    factual_without_nans = output_factuals
    counterfactuals_without_nans = output_counterfactuals

    columns = ["Distance_1", "Distance_2", "Distance_3", "Distance_4"]
        
    arr_f = ml_model.get_ordered_features(factual_without_nans).to_numpy()
    arr_cf = ml_model.get_ordered_features(
        counterfactuals_without_nans
    ).to_numpy()

    delta = arr_f - arr_cf 

    d1 = np.sum(np.invert(np.isclose(delta, np.zeros_like(delta))), axis=1, dtype=np.float).tolist()
    d1_old = np.sum(delta.round(2) != 0, axis=1, dtype=np.float).tolist()

    d2 = np.sum(np.abs(delta), axis=1, dtype=np.float).tolist()
    d3 = np.sum(np.square(np.abs(delta)), axis=1, dtype=np.float).tolist()

    results.append(pd.DataFrame({'L0': d1, 'L1': d2, 'L2': d3, 'time': df_cfs['time (seconds)'].mean()}))

temp = pd.concat(results)
temp.index = final_results.index
final_results = pd.concat([final_results, temp], axis=1)


/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change 

## Validity

In [8]:
results = []
for method in ['cchvae', 'cem-vae', 'revise', 'clue', 'crud', 'face']:
    if data_name == 'adult':
        cfs = pd.read_csv("Results/adult_manifold_results.csv")
        y_col = 'income'
    elif data_name == 'give_me_some_credit':
        cfs = pd.read_csv("Results/give_me_some_credit_manifold_results.csv")
        y_col = "SeriousDlqin2yrs"
    elif data_name == 'compas':
        cfs = pd.read_csv("Results/compas_manifold_results.csv")
        y_col = "score"

    cfs.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
    cfs.set_index(['index'], inplace=True)

    df_cfs = cfs[cfs['method'] == method].drop(['method',	'data'], axis=1)
    results.append(pd.DataFrame({'validity': df_cfs[y_col]}))

In [9]:
temp = ml_model.predict_proba(final_results)
temp2 = []
temp3 = []
for x in temp:
    temp2.append(x[1]>= 0.5) #  >= 0.5

    temp3.append(x[1]) #  >= 0.5
    
final_results['validity'] = temp2
final_results['prediction'] = temp3
final_results['validity'] = final_results['validity'].astype(int)

/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


## Feasibility

In [10]:
from sklearn.neighbors import NearestNeighbors

results = []
for method in ['cchvae', 'cem-vae', 'revise', 'clue', 'crud', 'face']:
    if data_name == 'adult':
        cfs = pd.read_csv("Results/adult_manifold_results.csv")
    elif data_name == 'give_me_some_credit':
        cfs = pd.read_csv("Results/give_me_some_credit_manifold_results.csv")
    elif data_name == 'compas':
        cfs = pd.read_csv("Results/compas_manifold_results.csv")
    factuals = predict_negative_instances(ml_model, dataset.df)
    test_factual = factuals.iloc[:100]

    cfs.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
    cfs.set_index(['index'], inplace=True)

    df_cfs = cfs[cfs['method'] == method].drop(['method',	'data'], axis=1)

    nan_idx = df_cfs.index[df_cfs.isnull().any(axis=1)]

    output_factuals = test_factual.copy()
    output_counterfactuals = df_cfs.copy()

    output_factuals = output_factuals.drop(index=nan_idx)
    output_counterfactuals = output_counterfactuals.drop(index=nan_idx)

    factual_without_nans = output_factuals
    counterfactuals_without_nans = output_counterfactuals


    cols = dataset.df.columns
    cols.drop(dataset.target)

    nbrs = NearestNeighbors(n_neighbors=5).fit(factual_without_nans[cols].values)

    for i, row in counterfactuals_without_nans[cols].iterrows():
        knn = nbrs.kneighbors(row.values.reshape((1, -1)), 5, return_distance=True)[0]
        
        results.append(np.mean(knn))
final_results['feasibility'] = results

/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change 

In [28]:
pd.set_option('display.max_columns', None)

temp = final_results[['L0', 'L1', 'L2',  'feasibility', 'violations', 'validity', 'prediction']]
cfs.dropna(inplace=True)
temp = pd.concat([temp, dataset.inverse_transform(cfs)], axis=1)


In [29]:
import pandas as pd

results = pd.read_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/Results/{data_name}_mcce_results_k_10000_n_100.csv", index_col=0)

results['data'] = data_name
results['method'] = 'mcce'
results.rename(columns={'violation': 'violations'}, inplace=True)

preds = ml_model.predict_proba(results)
new_preds = []
for x in preds:
    new_preds.append(x[1])
results['prediction'] = new_preds
results = dataset.inverse_transform(results)
results.head(1)

results['validity'] = np.where(np.asarray(new_preds) >= 0.5, 1, 0)

# results.loc[263]

/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


In [30]:
temp = pd.concat([temp, results[temp.columns]])

temp2 = factuals.copy()
preds = ml_model.predict_proba(temp2)
new_preds = []
for x in preds:
    new_preds.append(x[1])
temp2['prediction'] = new_preds
temp2 = dataset.inverse_transform(temp2)
temp2.head(1)
temp2['L0'] = np.nan
temp2['L1'] = np.nan
temp2['L2'] = np.nan
temp2['validity'] = np.nan
temp2['violations'] = np.nan
temp2['feasibility'] = np.nan
temp2['time (seconds)'] = np.nan
temp2['method'] = 'original'
temp2['data'] = data_name

temp = pd.concat([temp, temp2.iloc[0:100][temp.columns]], axis=0)
temp

/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


,L0,L1,L2,feasibility,violations,validity,prediction,method,data,age,two_year_recid,priors_count,length_of_stay,score,time (seconds),c_charge_degree,race,sex
24,6.0,2.472314,1.555753,0.782131,2.0,1.0,0.560921,cchvae,compas,44.282502,0.439713,11.041507,258.224912,1.0,11.609889,F,Other,Male
27,5.0,1.374783,0.518000,0.782488,1.0,1.0,0.559939,cchvae,compas,44.285066,0.439713,11.041029,258.882976,1.0,11.609889,F,Other,Male
33,6.0,2.544871,1.574804,0.782253,2.0,1.0,0.560327,cchvae,compas,44.281333,0.439724,11.043259,258.487816,1.0,11.609889,F,Other,Male
35,6.0,2.890713,1.866514,0.781968,3.0,1.0,0.561680,cchvae,compas,44.283871,0.439779,11.039302,257.871040,1.0,11.609889,F,Other,Male
40,6.0,2.485153,1.572486,0.782000,1.0,1.0,0.560917,cchvae,compas,44.281035,0.439820,11.045066,258.050512,1.0,11.609889,F,Other,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,NaN,NaN,NaN,NaN,NaN,NaN,0.315730,original,compas,26.000000,1.000000,4.000000,293.000000,1.0,NaN,M,Other,Male
728,NaN,NaN,NaN,NaN,NaN,NaN,0.453095,original,compas,24.000000,1.000000,5.000000,1.000000,1.0,NaN,F,African-American,Male
738,NaN,NaN,NaN,NaN,NaN,NaN,0.088032,original,compas,31.000000,1.000000,25.000000,117.000000,0.0,NaN,M,African-American,Male
740,NaN,NaN,NaN,NaN,NaN,NaN,0.419233,original,compas,39.000000,1.000000,13.000000,1.000000,0.0,NaN,F,African-American,Male


In [24]:
# results = pd.read_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/Results/{data_name}_baseline_results_n_100.csv", index_col=0)
# results['data'] = data_name
# results['method'] = 'baseline'
# results.rename(columns={'violation': 'violations'}, inplace=True)

# preds = ml_model.predict_proba(results)
# new_preds = []
# for x in preds:
#     new_preds.append(x[1])
# results['prediction'] = new_preds
# results = dataset.inverse_transform(results)
# results.head(1)

# results['validity'] = np.where(np.asarray(new_preds) >= 0.5, 1, 0)
# results[temp.columns]
# temp = pd.concat([temp, results[temp.columns]])
# temp

# temp.columns[9:13].to_list() + temp.columns[15:].to_list()

['age',
 'two_year_recid',
 'priors_count',
 'length_of_stay',
 'c_charge_degree',
 'race',
 'sex']

In [31]:
if data_name == 'give_me_some_credit':
    cols = ['method', 'data', 'prediction', 'L0', 'L1', 'L2', 'feasibility', 'violations', 'validity', 'time (seconds)'] + temp.columns[9:-1].to_list()
    temp = temp[cols]
elif data_name == 'adult':
    cols = ['method', 'data', 'prediction', 'L0', 'L1', 'L2', 'feasibility', 'violations', 'validity', 'time (seconds)'] + temp.columns[9:16].to_list() + temp.columns[17:].to_list()
    temp = temp[cols]

elif data_name == 'compas':
    cols = ['method', 'data', 'prediction', 'L0', 'L1', 'L2', 'feasibility', 'violations', 'validity', 'time (seconds)'] + temp.columns[9:13].to_list() + temp.columns[15:].to_list()
    temp = temp[cols]
temp.to_csv(f"Final_results/{data_name}_results_mcce_and_carla_K_10000_n_100.csv")



## To get Table 1

In [18]:
temp = pd.read_csv("Final_results/adult_results_mcce_and_carla_K_10000_n_100.csv", index_col=0)

to_write = temp[['method', 'L0', 'L2', 'feasibility', 'violations', 'validity', 'time (seconds)']].groupby(['method']).mean()

to_write.reset_index(inplace=True)

CE_N = temp.groupby(['method']).size().reset_index().rename(columns={0: 'CE_N'})
to_write = pd.concat([to_write, CE_N.CE_N], axis=1)

# to_write.sort_values(['method'], inplace=True, ascending=False)
to_write = to_write[['method', 'L0', 'L2', 'feasibility', 'violations', 'validity', 'CE_N', 'time (seconds)']]

print(to_write.to_latex(index=False, float_format="%.2f", ))

\begin{tabular}{lrrrrrrr}
\toprule
   method &    L0 &   L2 &  feasibility &  violations &  validity &  CE\_N &  time (seconds) \\
\midrule
   cchvae &  9.39 & 2.67 &         0.93 &        1.29 &      1.00 &   100 &           85.69 \\
  cem-vae &  6.52 & 2.19 &         0.98 &        1.04 &      0.52 &   100 &           81.22 \\
     clue & 11.34 & 2.00 &         1.26 &        1.29 &      1.00 &   100 &          354.79 \\
     crud & 10.66 & 2.33 &         1.09 &        1.29 &      1.00 &   100 &         1145.26 \\
     face &  6.96 & 2.72 &         1.03 &        1.36 &      1.00 &   100 &          955.89 \\
     mcce &  3.04 & 0.74 &         0.05 &        0.00 &      1.00 &   100 &          106.21 \\
 original &   NaN &  NaN &          NaN &         NaN &       NaN &   100 &             NaN \\
   revise &  9.66 & 3.22 &         1.06 &        1.39 &      1.00 &    38 &          907.06 \\
\bottomrule
\end{tabular}



In [19]:
temp = pd.read_csv("Final_results/give_me_some_credit_results_mcce_and_carla_K_10000_n_100.csv", index_col=0)

to_write = temp[['method', 'L0', 'L2', 'feasibility', 'violations', 'validity', 'time (seconds)']].groupby(['method']).mean()

to_write.reset_index(inplace=True)

CE_N = temp.groupby(['method']).size().reset_index().rename(columns={0: 'CE_N'})
to_write = pd.concat([to_write, CE_N.CE_N], axis=1)

# to_write.sort_values(['method'], inplace=True, ascending=False)
to_write = to_write[['method', 'L0', 'L2', 'feasibility', 'violations', 'validity', 'CE_N', 'time (seconds)']]

print(to_write.to_latex(index=False, float_format="%.2f", ))

\begin{tabular}{lrrrrrrr}
\toprule
   method &    L0 &   L2 &  feasibility &  violations &  validity &  CE\_N &  time (seconds) \\
\midrule
   cchvae & 10.00 & 0.41 &         0.44 &        1.00 &      1.00 &   100 &          202.02 \\
  cem-vae &  8.43 & 0.58 &         0.42 &        0.93 &      0.96 &   100 &           86.35 \\
     clue & 10.00 & 0.37 &         0.40 &        0.81 &      1.00 &   100 &          341.87 \\
     crud & 10.00 & 0.38 &         0.49 &        1.00 &      1.00 &   100 &         1153.13 \\
     face &  8.51 & 0.60 &         0.46 &        0.98 &      1.00 &   100 &         3052.45 \\
     mcce &  3.48 & 0.18 &         0.03 &        0.00 &      1.00 &   100 &         1242.18 \\
 original &   NaN &  NaN &          NaN &         NaN &       NaN &   100 &             NaN \\
   revise & 10.00 & 1.00 &         0.79 &        1.00 &      1.00 &   100 &          805.43 \\
\bottomrule
\end{tabular}



In [32]:
temp = pd.read_csv("Final_results/compas_results_mcce_and_carla_K_10000_n_100.csv", index_col=0)

to_write = temp[['method', 'L0', 'L2', 'feasibility', 'violations', 'validity', 'time (seconds)']].groupby(['method']).mean()

to_write.reset_index(inplace=True)

CE_N = temp.groupby(['method']).size().reset_index().rename(columns={0: 'CE_N'})
to_write = pd.concat([to_write, CE_N.CE_N], axis=1)

# to_write.sort_values(['method'], inplace=True, ascending=False)
to_write = to_write[['method', 'L0', 'L2', 'feasibility', 'violations', 'validity', 'CE_N', 'time (seconds)']]

print(to_write.to_latex(index=False, float_format="%.2f", ))

\begin{tabular}{lrrrrrrr}
\toprule
   method &   L0 &   L2 &  feasibility &  violations &  validity &  CE\_N &  time (seconds) \\
\midrule
   cchvae & 5.88 & 1.47 &         0.78 &        1.88 &      1.00 &   100 &           11.61 \\
  cem-vae & 3.85 & 1.04 &         0.47 &        1.41 &      0.93 &   100 &           83.40 \\
     clue & 4.66 & 0.98 &         0.69 &        1.58 &      1.00 &    92 &          300.90 \\
     crud & 6.81 & 1.38 &         0.94 &        1.85 &      1.00 &   100 &         1005.92 \\
     face & 4.35 & 1.74 &         0.70 &        1.66 &      1.00 &   100 &           41.60 \\
     mcce & 1.42 & 0.10 &         0.00 &        0.00 &      1.00 &   100 &         2633.84 \\
 original &  NaN &  NaN &          NaN &         NaN &       NaN &   100 &             NaN \\
   revise & 5.88 & 6.13 &         2.20 &        1.88 &      1.00 &   100 &          696.05 \\
\bottomrule
\end{tabular}



## To get Adult examples in table 2

In [20]:
import pandas as pd

temp = pd.read_csv("Final_results/adult_results_mcce_and_carla_K_10000_n_100.csv", index_col=0)

# cols = ['Method', 'Pred', 'Age', 'Work Class', 'FNLWGT', 'Educat.', 'Mar. Stat.', 'Relat.', 'Cap. Gain', 'Cap. Loss', 'Hr.', 'Co.']

# cols = ['method', 'prediction', 'age', 'workclass', 'fnlwgt', 'education-num', 'marital-status', 'relationship', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']

to_write = temp.loc[31]
# to_write.columns = cols
# to_write.sort_values(['Method'], inplace=True, ascending=False)
pd.set_option('display.max_columns', None)
to_write


,method,data,prediction,L0,L1,L2,feasibility,violations,validity,time (seconds),age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income,marital-status,native-country,occupation,race,relationship,sex,workclass
31,cchvae,adult,0.693449,11.0,4.691737,4.141263,0.929137,1.0,1.0,85.685519,38.764985,189644.462285,10.111844,3652.984470,172.409726,40.223185,1.000000,Married,US,Managerial-Specialist,White,Husband,Male,Private
31,cem-vae,adult,0.500259,7.0,2.410045,2.039343,1.113540,1.0,1.0,81.218853,23.000000,190709.000094,12.000000,10303.477725,0.000000,51.999999,1.000000,Non-Married,US,Other,White,Non-Husband,Male,Non-Private
31,clue,adult,0.773839,8.0,2.803834,2.141725,1.477626,1.0,1.0,354.790007,10.527289,398961.583832,9.570491,10724.602753,-62.028129,49.267808,1.000000,Non-Married,US,Other,White,Non-Husband,Male,Private
31,crud,adult,0.945782,13.0,3.415565,1.386162,1.207662,1.0,1.0,1145.259907,50.366662,147878.430972,16.521214,8414.476454,-30.494723,41.116929,1.000000,Married,US,Managerial-Specialist,White,Husband,Male,Private
31,face,adult,0.509658,8.0,3.497612,3.072894,0.492528,1.0,1.0,955.889991,35.000000,38948.000000,11.000000,3103.000000,0.000000,40.000000,1.000000,Married,US,Other,White,Husband,Male,Private
31,mcce,adult,0.928255,3.0,0.638007,0.178944,0.037504,0.0,1.0,106.208441,20.000000,188923.000000,10.000000,34095.000000,0.000000,20.000000,0.928255,Non-Married,US,Other,Non-White,Non-Husband,Male,Private
31,original,adult,0.015605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.000000,266015.000000,10.000000,0.000000,0.000000,44.000000,0.000000,Non-Married,US,Other,Non-White,Non-Husband,Male,Private


In [21]:
# print(to_write.Pred.to_latex(index=False, float_format="%.2f", ))
feature = 'marital-status'
dct = {'Married': 'M', 'Non-Married': 'NM'}
to_write[feature] = [dct[item] for item in to_write[feature]]

feature = 'native-country'
dct = {'Non-US': 'NUS', 'US': 'US'}
to_write[feature] = [dct[item] for item in to_write[feature]]

feature = 'occupation'
dct = {'Managerial-Specialist': 'MS', 'Other': 'O'}
to_write[feature] = [dct[item] for item in to_write[feature]]

feature = 'race'
dct = {'White': 'W', 'Non-White': 'NW'}
to_write[feature] = [dct[item] for item in to_write[feature]]

feature = 'relationship'
dct = {'Husband': 'H', 'Non-Husband': 'NH'}
to_write[feature] = [dct[item] for item in to_write[feature]]

feature = 'sex'
dct = {'Male': 'M'}
to_write[feature] = [dct[item] for item in to_write[feature]]


feature = 'workclass'
dct = {'Self-emp-not-inc': 'SENI', 'Private': 'P', 'Non-Private': 'NP'}
to_write[feature] = [dct[item] for item in to_write[feature]]

/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [22]:
cols = ['method', 'age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', \
       'hours-per-week', 'marital-status', 'native-country', \
       'occupation', 'race', 'relationship', 'sex', 'workclass']

print(to_write[cols].to_latex(index=False, float_format="%.0f", ))

\begin{tabular}{lrrrrrrlllllll}
\toprule
   method &  age &  fnlwgt &  education-num &  capital-gain &  capital-loss &  hours-per-week & marital-status & native-country & occupation & race & relationship & sex & workclass \\
\midrule
   cchvae &   39 &  189644 &             10 &          3653 &           172 &              40 &              M &             US &         MS &    W &            H &   M &         P \\
  cem-vae &   23 &  190709 &             12 &         10303 &             0 &              52 &             NM &             US &          O &    W &           NH &   M &        NP \\
     clue &   11 &  398962 &             10 &         10725 &           -62 &              49 &             NM &             US &          O &    W &           NH &   M &         P \\
     crud &   50 &  147878 &             17 &          8414 &           -30 &              41 &              M &             US &         MS &    W &            H &   M &         P \\
     face &   35 &   38948 &  

## To get GMC examples in table 3

In [24]:
temp = pd.read_csv("Final_results/give_me_some_credit_results_mcce_and_carla_K_10000_n_100.csv", index_col=0)
# temp.loc[263]

cols = ['method', 'prediction', 'age', 'RevolvingUtilizationOfUnsecuredLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents']

to_write = temp[cols].loc[263]

cols = ['Method', 'Pred', 'Age', 'Unsec. Lines', 'Nb Days Past 30', 'Debt Ratio', 'Month Inc.', 'Nb Credit Lines', 'Nb Times 90 Days Late', 'Nb Real Estate Loans', 'Nb Times 60 Days Past', 'Nb Dep.']

to_write.columns = cols
# to_write.sort_values(['Method'], inplace=True, ascending=False)

# print(to_write.to_latex(index=False, float_format="%.0f", ))

print(to_write.to_latex(index=False, float_format="%.2f", ))

\begin{tabular}{lrrrrrrrrrrr}
\toprule
   Method &  Pred &   Age &  Unsec. Lines &  Nb Days Past 30 &  Debt Ratio &  Month Inc. &  Nb Credit Lines &  Nb Times 90 Days Late &  Nb Real Estate Loans &  Nb Times 60 Days Past &  Nb Dep. \\
\midrule
   cchvae &  0.96 & 51.23 &          4.18 &             0.27 &        9.33 &     6247.85 &             8.49 &                   0.11 &                  0.98 &                   0.09 &     0.82 \\
  cem-vae &  0.60 & 69.00 &          0.00 &             2.00 &        0.19 &     3000.00 &            10.00 &                   0.00 &                  0.00 &                   2.00 &     0.00 \\
     clue &  1.00 & 37.31 &         -2.75 &             1.99 &     -148.85 &      627.71 &             8.41 &                  -1.91 &                  0.27 &                  -1.19 &     2.14 \\
     crud &  1.00 & 82.80 &          0.02 &             0.52 &       88.25 &     9245.43 &            17.24 &                  -1.01 &                  1.14 &          

In [29]:
if data_name == 'give_me_some_credit':
    features = [ 'age', 'RevolvingUtilizationOfUnsecuredLines', 'NumberOfTime30-59DaysPastDueNotWorse','DebtRatio', 'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents']
    metric_names = ['method', 'L0', 'L1', 'violations', 'validity', 'prediction']

    temp = dataset.inverse_transform(final_results.dropna()[features])
    temp = pd.concat([final_results[metric_names], temp], axis=1)
    # temp.sort_values(temp.index.name)

In [31]:
if data_name == 'give_me_some_credit':
    mcce_results = pd.read_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/give_me_some_credit_mcce_results_k_10000.csv")
    mcce_results.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
    mcce_results.set_index(['index'], inplace=True)

    predictions = ml_model.predict_proba(mcce_results)
    temp3 = []
    for x in predictions:
        temp3.append(x[1]) #  >= 0.5
        
    # temp.index = final_results.index
    mcce_results['prediction'] = temp3


In [33]:
if data_name == 'give_me_some_credit':
    mcce_results.sort_values(mcce_results.index.name, inplace=True)
    mcce_results['method'] = 'mcce'
    mcce_results.rename(columns={'success': 'validity', 'violation': 'violations'}, inplace=True)
    temp_mcce = dataset.inverse_transform(mcce_results.dropna()[features])
    temp_mcce = pd.concat([mcce_results[metric_names], temp_mcce], axis=1)
    # temp_mcce.sort_values(temp_mcce.index.name)

In [34]:
if data_name == 'give_me_some_credit':
    pd.set_option('display.max_columns', None)
    temp2 = pd.concat([temp, temp_mcce], axis=0)

    temp2.sort_values(temp2.index.name)

    features = ['method', 'prediction', 'age', 'RevolvingUtilizationOfUnsecuredLines', 'NumberOfTime30-59DaysPastDueNotWorse','DebtRatio', 'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents']

    temp2.loc[263][features]

In [35]:
if data_name == 'give_me_some_credit':
    factuals = predict_negative_instances(ml_model, dataset.df)
    dataset.inverse_transform(factuals.iloc[3:4])[['age', 'RevolvingUtilizationOfUnsecuredLines', 'NumberOfTime30-59DaysPastDueNotWorse','DebtRatio', 'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents']]

    ml_model.predict_proba(factuals.iloc[3:4])


In [34]:
temp = pd.read_csv("Final_results/compas_results_mcce_and_carla_K_10000_n_100.csv", index_col=0)
temp

,method,data,prediction,L0,L1,L2,feasibility,violations,validity,time (seconds),age,two_year_recid,priors_count,length_of_stay,c_charge_degree,race,sex
24,cchvae,compas,0.560921,6.0,2.472314,1.555753,0.782131,2.0,1.0,11.609889,44.282502,0.439713,11.041507,258.224912,F,Other,Male
27,cchvae,compas,0.559939,5.0,1.374783,0.518000,0.782488,1.0,1.0,11.609889,44.285066,0.439713,11.041029,258.882976,F,Other,Male
33,cchvae,compas,0.560327,6.0,2.544871,1.574804,0.782253,2.0,1.0,11.609889,44.281333,0.439724,11.043259,258.487816,F,Other,Male
35,cchvae,compas,0.561680,6.0,2.890713,1.866514,0.781968,3.0,1.0,11.609889,44.283871,0.439779,11.039302,257.871040,F,Other,Male
40,cchvae,compas,0.560917,6.0,2.485153,1.572486,0.782000,1.0,1.0,11.609889,44.281035,0.439820,11.045066,258.050512,F,Other,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,original,compas,0.315730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.000000,1.000000,4.000000,293.000000,M,Other,Male
728,original,compas,0.453095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.000000,1.000000,5.000000,1.000000,F,African-American,Male
738,original,compas,0.088032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.000000,1.000000,25.000000,117.000000,M,African-American,Male
740,original,compas,0.419233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.000000,1.000000,13.000000,1.000000,F,African-American,Male


In [40]:

cols = ['method', 'prediction', 'age', 'two_year_recid', 'priors_count', 'length_of_stay', 'c_charge_degree', 'race', 'sex']

to_write = temp[cols].loc[40]

cols = ['Method', 'Pred', 'Age', 'Two Year Recid', 'Priors Count', 'Length of Stay', 'C Charge Degree', 'Race', 'Sex']

to_write.columns = cols
# to_write.sort_values(['Method'], inplace=True, ascending=False)

# print(to_write.to_latex(index=False, float_format="%.0f", ))

print(to_write.to_latex(index=False, float_format="%.2f", ))

round(to_write[['Method', 'Pred']],2)

\begin{tabular}{lrrrrrlll}
\toprule
   Method &  Pred &    Age &  Two Year Recid &  Priors Count &  Length of Stay & C Charge Degree &              Race &   Sex \\
\midrule
   cchvae &  0.56 &  44.28 &            0.44 &         11.05 &          258.05 &               F &             Other &  Male \\
  cem-vae &  0.96 &  57.00 &            1.00 &          0.00 &           -1.00 &               F &             Other &  Male \\
     clue &  0.83 &  30.59 &            0.87 &          9.14 &         -193.67 &               M &             Other &  Male \\
     crud &  1.00 & 118.66 &            0.91 &          2.00 &          386.79 &               F &             Other &  Male \\
   revise &  0.80 &  47.47 &            0.46 &          9.40 &          210.84 &               F &             Other &  Male \\
     face &  0.53 &  23.00 &            1.00 &          3.00 &            6.00 &               F &  African-American &  Male \\
     mcce &  0.62 &  25.00 &            1.00 &          6.0

,Method,Pred
40,cchvae,0.56
40,cem-vae,0.96
40,clue,0.83
40,crud,1.00
40,revise,0.80
40,face,0.53
40,mcce,0.62
40,original,0.43
